# Load dependencies

In [ ]:
!pip install autogluon
!pip install --upgrade ipykernel

In [ ]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor
from autogluon.tabular.models.knn.knn_rapids_model import KNNRapidsModel
from autogluon.tabular.models.lr.lr_rapids_model import LinearRapidsModel

In [ ]:
train_data=pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv')
test_data=pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv')
sub=pd.read_csv('../input/tabular-playground-series-jun-2021/sample_submission.csv')

In [ ]:
for col in train_data.columns: train_data[col] = train_data[col].astype('category')
for col in test_data.columns: test_data[col] = test_data[col].astype('category')

# AutoGluon

see: 

- https://auto.gluon.ai/stable/tutorials/tabular_prediction/tabular-kaggle.html
- https://www.kaggle.com/innixma/autogluon-rapids-top-1

In [ ]:
predictor = TabularPredictor(label="target",
                             problem_type='multiclass',
                             eval_metric="log_loss",
                             learner_kwargs={'ignored_columns': ['id']},
                             path=".",
                             verbosity=3)

predictor.fit(
    train_data, 
    presets='best_quality',
    hyperparameters={
        KNNRapidsModel: {},
        LinearRapidsModel: {},
        'NN': {},
        'RF': {},
        'XGB': {'ag_args_fit': {'num_gpus': 1}},
        'CAT': {'ag_args_fit': {'num_gpus': 1}},
        'GBM': [{}, {'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, 'GBMLarge'],
        'NN': {'ag_args_fit': {'num_gpus': 1}},
        'FASTAI': {'ag_args_fit': {'num_gpus': 1}},
    },
time_limit=60*60*8)

results = predictor.fit_summary()

# Sub

In [ ]:
sub.iloc[:, 1:] = predictor.predict_proba(test_data)
sub.to_csv("sub_autogluon.csv", index=False)

In [ ]:
sub